# Manejo de Datos en PyTorch

## Librerías

In [17]:
import csv
import functools
import gzip
import numpy as np
import pandas as pd
import torch
import json
import tempfile

from gensim.models import KeyedVectors
from gensim.parsing import preprocessing
from torch.utils.data import Dataset, DataLoader, IterableDataset

## La clase Dataset

La clase abstracta [`torch.utils.data.Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) es la clase base para construir un dataset de PyTorch. Cualquier dataset personalizado debe heredar de dicha clase e implementar los siguientes métodos:

- `__len__`: Para que `len(dataset)` devuelva el tamaño del conjunto de datos.
- `__getitem__`: Para soportar indexado de manera que `dataset[i]` devuelva el elemento `i`. Es común que en ciertos casos se utilice este método para levantar el dato real (e.g. una imagen) mientras que lo que se guarde en el dataset sea sólo una referencia a dicho dato (e.g. un path a la imagen). De esta manera se evita cargar muchas imágenes en memoria, haciendo que sea menos demandante a nivel RAM.

In [2]:
class IMDBReviewsDataset(Dataset):
    def __init__(self, path, transform=None):
        self.dataset = pd.read_csv(path)
        self.transform = transform
    
    def __len__(self):
        return self.dataset.shape[0]

    def __getitem__(self, item):
        if torch.is_tensor(item):
            item = item.tolist()  # Deal with list of items instead of tensor
        
        item = {
            "data": self.dataset.iloc[item]["review"],
            "target": self.dataset.iloc[item]["sentiment"]
        }

        if self.transform:
            item = self.transform(item)
        
        return item

dataset = IMDBReviewsDataset("./data/imdb_reviews.csv.gz")
print(f"Dataset loaded with {len(dataset)} elements")
print(f"Sample element:\n{dataset[0]}")

Dataset loaded with 50000 elements
Sample element:
{'data': "One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><b

## Transformaciones

El ejemplo anterior nos muestra el uso básico, pero claramente no podemos pasarle eso a una red neuronal, no puede manejar texto. Es para eso que tenemos que hacer algún tipo de transformación sobre los atributos (en este caso el único atributo es el texto). 

### Normalización

En particular, como vemos en el caso anterior, el texto no está normalizado, parte de las transformaciones pueden incluir realizar algún tipo de normalización. Para eso hagamos uso de [`gensim`](https://radimrehurek.com/gensim/index.html), en particular utilizaremos el módulo [`preprocessing`](https://radimrehurek.com/gensim/parsing/preprocessing.html#module-gensim.parsing.preprocessing) que se encargará de hacer varias normalizaciones por defecto

In [3]:
class TextPreprocess:
    def __init__(self, filters=None):
        if filters:
            self.filters = filters
        else:
            self.filters = [
                lambda s: s.lower(),
                preprocessing.strip_tags,
                preprocessing.strip_punctuation,
                preprocessing.strip_multiple_whitespaces,
                preprocessing.strip_numeric,
                preprocessing.remove_stopwords,
                preprocessing.strip_short,
            ]
        
    def _preprocess_string(self, string):
        return preprocessing.preprocess_string(string, filters=self.filters)

    def _encode_target(self, target):
        return 1 if target == "positive" else 0

    def __call__(self, item):
        if isinstance(item["data"], str):
            data = self._preprocess_string(item["data"])
        else:
            data = [self._preprocess_string(d) for d in item["data"]]
        
        if isinstance(item["target"], str):
            target = self._encode_target(item["target"])
        else:
            target = [self._encode_target(t) for t in item["target"]]
        
        return {
            "data": data,
            "target": target
        }

preprocess = TextPreprocess()
print(preprocess(dataset[0]))

{'data': ['reviewers', 'mentioned', 'watching', 'episode', 'hooked', 'right', 'exactly', 'happened', 'thing', 'struck', 'brutality', 'unflinching', 'scenes', 'violence', 'set', 'right', 'word', 'trust', 'faint', 'hearted', 'timid', 'pulls', 'punches', 'regards', 'drugs', 'sex', 'violence', 'hardcore', 'classic', 'use', 'word', 'called', 'nickname', 'given', 'oswald', 'maximum', 'security', 'state', 'penitentary', 'focuses', 'mainly', 'emerald', 'city', 'experimental', 'section', 'prison', 'cells', 'glass', 'fronts', 'face', 'inwards', 'privacy', 'high', 'agenda', 'city', 'home', 'aryans', 'muslims', 'gangstas', 'latinos', 'christians', 'italians', 'irish', 'scuffles', 'death', 'stares', 'dodgy', 'dealings', 'shady', 'agreements', 'far', 'away', 'main', 'appeal', 'fact', 'goes', 'shows', 'wouldn', 'dare', 'forget', 'pretty', 'pictures', 'painted', 'mainstream', 'audiences', 'forget', 'charm', 'forget', 'romance', 'mess', 'episode', 'saw', 'struck', 'nasty', 'surreal', 'couldn', 'ready',

### Conversión a vectores

Podemos continuar convertiendo el texto en una representación por vectores. Si bien hay muchas posibilidades (siendo la bolsa de palabras una de las más utilizadas), en general para Deep Learning se prefieren representaciones utilizando vectores contínuos, obtenidos por algún método del estilo de Word2Vec, Glove o FastText. Para este caso utilizaremos las representaciones de Glove de dimensión 50 que se dejaron para descargar en el [notebook 0](./0_set_up.ipynb).

In [5]:
class VectorizeText:
    def __init__(self, glove_vectors_path):
        self.glove_model = KeyedVectors.load_word2vec_format(glove_vectors_path,
                                                             binary=False, no_header=True)
        self.unkown_vector = np.random.randn(self.glove_model.vector_size)  # Random vector for unknown words
    
    def _get_vector(self, word):
        if word in self.glove_model:
            return self.glove_model[word]
        else:
            return self.unkown_vector
    
    def _get_vectors(self, sentence):
        return np.vstack([self._get_vector(word) for word in sentence])
    
    def __call__(self, item):
        review = []
        if isinstance(item["data"][0], str):
            review = self._get_vectors(item["data"])
        else:
            review = [self._get_vectors(d) for d in item["data"]]

        return {
            "data": review,
            "target": item["target"]
        }

vectorizer = VectorizeText("./data/glove.6B.50d.txt.gz")
vectorizer(preprocess(dataset[0]))

{'data': array([[-0.18105   , -0.79229999, -0.097616  , ...,  1.42859995,
         -0.032471  ,  0.47235999],
        [ 0.69395   ,  0.69261003, -0.21608   , ...,  0.2247    ,
         -0.23197   ,  0.0062523 ],
        [-0.0049087 ,  0.12611   ,  0.14056   , ..., -0.58464003,
         -0.31830999,  0.31564   ],
        ...,
        [ 0.25435999, -0.44304001, -0.12524   , ...,  0.73352998,
          0.026198  ,  0.30408001],
        [-0.058468  ,  0.019087  ,  0.089056  , ..., -0.28176001,
          0.045137  , -0.18802001],
        [ 0.14443   ,  0.39103001, -0.93454999, ..., -0.71325999,
         -0.54575998,  0.13952   ]]),
 'target': 1}

### Combinación de vectores

Si bien ahora estamos con una versión de los atributos que podría pasar por una red neuronal, hay un problema, las distintas reviews tienen largo distinto y como el algoritmo se entrena en lotes (*mini-batches*) estas requieren tener todas el mismo largo. Hay varias maneras de lidiar con esto, cada una con sus ventajas y desventajas. Dado que por ahora solo vimos perceptrón multicapa, que espera algo de tamaño fijo, una opción sencilla puede ser la de simplemente promediar los vectores de palabras.

In [6]:
class WordVectorsAverage:
    def __call__(self, item):
        if item["data"][0].ndim == 2:
            data = np.vstack([np.mean(d, axis=0) for d in item["data"]])
        else:
            data = np.mean(item["data"], axis=0)
        
        return {
            "data": data,
            "target": item["target"]
        }

vector_average = WordVectorsAverage()
vector_average(vectorizer(preprocess(dataset[0])))

{'data': array([ 0.10920768,  0.0152441 , -0.1070746 , -0.28202113,  0.28231286,
         0.0607865 , -0.05840408, -0.05096249, -0.14460429,  0.20120667,
        -0.20296218, -0.11786137, -0.14528444, -0.00339705,  0.2720593 ,
         0.02072164,  0.02963385, -0.00692017, -0.124135  , -0.26061848,
        -0.06165497,  0.30771325,  0.14370682,  0.20214674,  0.12474517,
        -1.09261586, -0.35921837,  0.1742825 ,  0.42563727, -0.16268977,
         2.04543871,  0.1356153 , -0.038627  , -0.39735188, -0.07564671,
         0.10941898, -0.07223698, -0.16656189, -0.20611945, -0.23766232,
        -0.11803514,  0.06924455,  0.05824452,  0.26010405,  0.13715907,
         0.00354383,  0.01808459, -0.15028533, -0.03066247, -0.0714772 ]),
 'target': 1}

### Conversión de vectores a tensores

En el paso final, debemos convertir nuestros datos de arrays de `numpy` a tensores de PyTorch.

In [7]:
class ToTensor:
    def __call__(self, item):
        """
        This espects a single array.
        """
        return {
            "data": torch.from_numpy(item["data"]),
            "target": torch.tensor(item["target"])
        }

to_tensor = ToTensor()
to_tensor(vector_average(vectorizer(preprocess(dataset[0]))))

{'data': tensor([ 0.1092,  0.0152, -0.1071, -0.2820,  0.2823,  0.0608, -0.0584, -0.0510,
         -0.1446,  0.2012, -0.2030, -0.1179, -0.1453, -0.0034,  0.2721,  0.0207,
          0.0296, -0.0069, -0.1241, -0.2606, -0.0617,  0.3077,  0.1437,  0.2021,
          0.1247, -1.0926, -0.3592,  0.1743,  0.4256, -0.1627,  2.0454,  0.1356,
         -0.0386, -0.3974, -0.0756,  0.1094, -0.0722, -0.1666, -0.2061, -0.2377,
         -0.1180,  0.0692,  0.0582,  0.2601,  0.1372,  0.0035,  0.0181, -0.1503,
         -0.0307, -0.0715], dtype=torch.float64),
 'target': tensor(1)}

### Componiendo las transformaciones

Para evitar tener que llamar a todas las funciones de transformación que querramos aplicar, para ello hacemos uso del parámetro `transform` que definimos en nuestro `Dataset` y un poco de ayuda de `functools`.

In [8]:
def compose(*functions):
    return functools.reduce(lambda f, g: lambda x: g(f(x)), functions, lambda x: x)

dataset = IMDBReviewsDataset("./data/imdb_reviews.csv.gz",
                             transform=compose(preprocess, vectorizer, vector_average, to_tensor))
print(f"Dataset loaded with {len(dataset)} elements")
print(f"Sample element:\n{dataset[0]}")

Dataset loaded with 50000 elements
Sample element:
{'data': tensor([ 0.1092,  0.0152, -0.1071, -0.2820,  0.2823,  0.0608, -0.0584, -0.0510,
        -0.1446,  0.2012, -0.2030, -0.1179, -0.1453, -0.0034,  0.2721,  0.0207,
         0.0296, -0.0069, -0.1241, -0.2606, -0.0617,  0.3077,  0.1437,  0.2021,
         0.1247, -1.0926, -0.3592,  0.1743,  0.4256, -0.1627,  2.0454,  0.1356,
        -0.0386, -0.3974, -0.0756,  0.1094, -0.0722, -0.1666, -0.2061, -0.2377,
        -0.1180,  0.0692,  0.0582,  0.2601,  0.1372,  0.0035,  0.0181, -0.1503,
        -0.0307, -0.0715], dtype=torch.float64), 'target': tensor(1)}


### Iterando el dataset

Ya tenemos nuestro conjunto de datos con sus respectivas transformaciones. ¿Para qué nos sirve esto? Una opción es simplemente iterar en el conjunto de datos de a un elemento. Esto es sencillo, simplemente se hace a través de un `for`.

In [9]:
for idx, sample in enumerate(dataset):
    print(sample["data"])
    print(sample["target"])
    print("=" * 50)
    
    if idx == 2:
        break

tensor([ 0.1092,  0.0152, -0.1071, -0.2820,  0.2823,  0.0608, -0.0584, -0.0510,
        -0.1446,  0.2012, -0.2030, -0.1179, -0.1453, -0.0034,  0.2721,  0.0207,
         0.0296, -0.0069, -0.1241, -0.2606, -0.0617,  0.3077,  0.1437,  0.2021,
         0.1247, -1.0926, -0.3592,  0.1743,  0.4256, -0.1627,  2.0454,  0.1356,
        -0.0386, -0.3974, -0.0756,  0.1094, -0.0722, -0.1666, -0.2061, -0.2377,
        -0.1180,  0.0692,  0.0582,  0.2601,  0.1372,  0.0035,  0.0181, -0.1503,
        -0.0307, -0.0715], dtype=torch.float64)
tensor(1)
tensor([ 0.0890,  0.1748, -0.3817, -0.1365,  0.3764,  0.1839, -0.2633, -0.1469,
        -0.1334,  0.3825, -0.0662,  0.2222, -0.1418,  0.1494,  0.2704, -0.0477,
         0.1079,  0.1082, -0.1205, -0.3886,  0.0806,  0.3587,  0.0260,  0.0627,
         0.4123, -0.7971, -0.5830,  0.1019,  0.3950, -0.1518,  2.1207, -0.1356,
         0.1156, -0.4135,  0.0262,  0.3190, -0.0536,  0.2520, -0.2277, -0.3254,
         0.1563,  0.1237, -0.1276,  0.0428, -0.0289,  0.0723, 

## La clase Dataloader

El problema con iterar de a un elemento es que estamos limitados al querer entrenar un modelo. Por empezar, los modelos de Deep Learning suelen ser más eficientes si se entrenan utilizando algún tipo de entrenamiento por *mini-batches*. Además, hay otras cosas como mezclar los elementos (*shuffling*) o cargar datos en paralelo vía distintos *multiprocess workers*. La clase [`torch.utils.data.DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) precisamente se encarga de hacer eso por nosotros:

In [10]:
dataloader = DataLoader(
    dataset,
    batch_size=4,
    shuffle=True,
    num_workers=2
)

for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, 
          sample_batched['data'].size(),
          sample_batched['target'].size())

    if i_batch == 2:
        print(sample_batched["data"])
        print(sample_batched["target"])
        break

0 torch.Size([4, 50]) torch.Size([4])
1 torch.Size([4, 50]) torch.Size([4])
2 torch.Size([4, 50]) torch.Size([4])
tensor([[ 3.4637e-01,  2.0785e-01, -2.9570e-01,  5.4333e-02,  4.0412e-01,
          1.9114e-01, -1.4682e-01, -2.3888e-01, -1.7727e-01,  3.4228e-01,
         -2.9474e-01,  2.6088e-01, -4.2572e-01,  3.2696e-01,  5.1748e-01,
         -9.1188e-02,  2.7123e-01,  2.7787e-01, -3.0447e-01, -1.9255e-01,
          6.1705e-02,  2.6281e-01, -2.4151e-02,  2.9278e-01,  4.6881e-01,
         -1.1142e+00, -7.1383e-01,  1.3878e-01,  4.5731e-01, -1.6761e-02,
          2.2293e+00, -1.9096e-01,  9.3975e-02, -3.6185e-01,  3.9700e-03,
          3.1308e-01, -4.8333e-02,  2.4677e-02, -5.0735e-01, -5.9055e-01,
          9.7112e-04,  2.2367e-01,  1.2639e-03, -2.2762e-01, -2.5028e-01,
          9.0740e-02,  1.9136e-01, -2.8286e-01,  1.2155e-01,  2.7905e-01],
        [ 1.0193e-01,  9.7284e-02, -2.0748e-01, -2.1992e-01,  3.3393e-01,
          1.7484e-01, -3.6629e-01,  4.2254e-02, -7.9503e-02,  3.5214e-0

## La clase IterableDataset

El método preferido para trabajar con conjuntos de datos en PyTorch es `torch.utils.data.Dataset`. En general, hacer uso inteligente del método `__getitem__`, e.g. usándolo para cargar imágenes a medida que sean necesitadas y no al instanciar el dataset, es la mejor manera de trabajar con un conjunto de datos. En particular, de esta forma es mucho más fácil hacer *shuffling* de los datos y demás. No obstante, no siempre esto es posible, muchas veces el conjunto de datos es demasiado grande para levantarlo en memoria (aunque sólo levantemos referencias). Para esos casos, PyTorch ofrece la clase [`torch.utils.data.IterableDataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset), en este caso el único método que es requerido implementar es `__iter__`.

In [19]:
class MeLiChallengeDataset(IterableDataset):
    def __init__(self, path, transform=None):
        self.dataset_path = path
        self.transform = transform

    def __iter__(self):
        with gzip.open(self.dataset_path, "rt") as fh:
            for l in fh:
                data = json.loads(l)
                item = {
                    "data": data['title'],
                    "target": data['category']
                }
                
                if self.transform:
                    yield self.transform(item)
                else:
                    yield item

dataset = MeLiChallengeDataset("./data/meli-challenge-2019/spanish.train.jsonl.gz")
dataloader = DataLoader(dataset, batch_size=4, shuffle=False, num_workers=0)
dataiter = iter(dataloader)
print(f"Sample batch:\n{dataiter.next()}")

Sample batch:
{'data': ['Casita Muñecas Barbies Pintadas', 'Neceser Cromado Holográfico ', 'Funda Asiento A Medida D20 Chevrolet', 'Embrague Ford Focus One 1.8 8v Td (90cv) Desde 01-99'], 'target': ['DOLLHOUSES', 'TOILETRY_BAGS', 'CAR_SEAT_COVERS', 'AUTOMOTIVE_CLUTCH_KITS']}
